In [4]:
#HIDDEN
#libraries

import time
import scipy
import matplotlib.pyplot as plt
import matplotlib
import sys
import itertools
import json
import numpy as np
from ipywidgets import interactive
from ipywidgets import Layout, Button, Box, VBox, HBox, GridBox, Label
import ipywidgets as widgets
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings('ignore')
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In an ongoing series of field experiments in the Dickinson Lab, a large mass of flies is released in an open desert, with a circular array of odor-baited traps positioned at 1 km distance. The traps release odor plumes, which the flies then encounter and chase up to their source.

In this widget, we explored how various factors such as the windspeed, the odor plume shape, and the flight strategy of the flies affected how they arrive at the collection traps.

In [2]:
# functions for calculations


def create_circle_of_sources(number_sources,radius,strength):
    location_list = []
    for i in range(number_sources):
        angle = i*(2.0*scipy.pi)/number_sources
        x = radius*scipy.cos(angle)
        y = radius*scipy.sin(angle)
        location_list.append((x,y))
    strength_list = [strength for x in location_list]
    return location_list, strength_list

def sigmoid(x,x_0,L,y_0,k):
    return (x-x_0)+(L/2)+y_0 - L/(np.exp(-k*(x-x_0))+1)

def find_nearest(array, value):
    #For each element in value, returns the index of array it is closest to.
    #array should be 1 x n and value should be m x 1
    idx = (np.abs(array - value)).argmin(axis=1) #this rounds up and down (
    #of the two values in array closest to value, picks the closer. (not the larger or the smaller)
    return idx

def sigmoidm(x,x_0,L,y_0,k,m):
    return m*(x-x_0)+(L/2)+y_0 - L/(np.exp(-k*(x-x_0))+1)

def speed_sigmoid_func_kwarg(x,x_0a=-0.4,x_0b=3.6,
    L=0.8,k=4.,y_0=1.6,m=1.):
    output = np.zeros_like(x)
    output[(x>=x_0a)&(x<=x_0b)] = y_0
    output[x<x_0a] = sigmoidm(x[x<x_0a],x_0a,L,y_0,k,m)
    output[x>x_0b] = sigmoidm(x[x>x_0b],x_0b,L,y_0,k,m)
    return output

def f0(intended_heading_angles,wind_mag,wind_angle,
    x_0a,x_0b,L,y_0,plot=False,c1=0,c2=1):
    
    #Converts intended heading angles to track heading angles
    #Currently only have computation for c1 = 0, c2 = 1
    n = len(intended_heading_angles)
    signed_wind_par_mags = wind_mag*np.cos(wind_angle-intended_heading_angles)
    adjusted_mag = speed_sigmoid_func_kwarg(signed_wind_par_mags,x_0a,x_0b,L,4.,y_0)
    
    intended_heading_unit_vectors = np.vstack((
        np.cos(intended_heading_angles),np.sin(intended_heading_angles)))
    intended_heading_vectors = adjusted_mag*intended_heading_unit_vectors

    w_vec = np.array([wind_mag*np.cos(wind_angle),wind_mag*np.sin(wind_angle)])
    wind_par_vec = (np.dot(
        w_vec,intended_heading_unit_vectors))*intended_heading_unit_vectors

    w_perp_vec = w_vec[:,None] - wind_par_vec
    heading_final_vec = intended_heading_vectors+c1*wind_par_vec+c2*w_perp_vec
    dispersing_speeds = np.sqrt(np.sum(heading_final_vec**2,axis=0))
    track_heading_angles = np.arctan2(heading_final_vec[1],heading_final_vec[0])

    return track_heading_angles,dispersing_speeds


def f1(track_heading_angles,source_locations,wind_angle,release_location=np.array([0,0])):
    #cone_angle: angle of spread from directly downwind
    #ie cone_angle = 10 degrees gives a 20 degree wedge

    #Convert track_heading_angles to a list of plume distances for each fly
    #(for example, the row for one fly would be [np.nan,500,700,800,np.nan,np.nan,np.nan,np.nan]
    # if the fly's future path intersected only plumes 2,3,4, at those distances) )

    #Using geometric algorithm here: http://geomalgorithms.com/a05-_intersect-1.html
    #referring to diagram under 'Non-Parallel Lines'

    #u - vector from source (P_0) downwind
    #v - heading vector originating from release location (Q_0)
    #w - vector from release location (Q_0) to source (P_0)
    #s_1 - distance along u from source (P_0) to intersection point
    #t_1 - distance along v from release_location (Q_0) to intersection point


    #Turn the track heading angles into unit vectors (v)
    fly_unit_vectors = np.vstack((np.cos(track_heading_angles),np.sin(track_heading_angles)))
    #shape is (2 x flies)

    #Turn the wind angle into a plume unit vector (u)
    plume_unit_vector = np.array([np.cos(wind_angle),np.sin(wind_angle)])
    #shape is (2)



    #Compute vector from release location to each source
    w = source_locations - release_location
    #shape is (traps x 2)

    #Compute s_1 & t_1

    #s_1 = (v2*w1-v1*w2)/(v1*u2-v2*u1)
    s_1 = (fly_unit_vectors[None,1,:]*w[:,0,None]-fly_unit_vectors[None,0,:]*w[:,1,None])/(
        fly_unit_vectors[None,0,:]*plume_unit_vector[None,1,None]-
            fly_unit_vectors[None,1,:]*plume_unit_vector[None,0,None])

    #t_1 = (u1*w2-u2*w1)/(u1*v2-u2*v1)
    t_1 = (plume_unit_vector[None,0,None]*w[:,1,None]-plume_unit_vector[None,1,None]*w[:,0,None])/(
        fly_unit_vectors[None,1,:]*plume_unit_vector[None,0,None]-
            fly_unit_vectors[None,0,:]*plume_unit_vector[None,1,None])

    #collapse extra axis
    s_1 = np.squeeze(s_1)

    #set all locations where s_1 or t_1 is negative to np.nan

    s_1[s_1<0.] = np.nan
    s_1[t_1<0.] = np.nan

    return s_1.T,t_1.T
    #s_1.T - shape is (n flies x n plumes), entry is the distance downwind of that plume
    #t_1.T - shape is (n flies x n plumes),
        #entry is the distance traveled from the release point to the intersection point

def f1_wedge(track_heading_angles,source_locations,wind_angle,
    cone_angle,release_location=np.array([0,0])):
    #Finds the intersection distances with edges of a wedge,
    #rather than a plume line

    #cone_angle: angle of spread from directly downwind
    #ie cone_angle = 10 degrees gives a 20 degree wedge

    #Convert track_heading_angles to a list of plume distances for each fly
    #(for example, the row for one fly would be [np.nan,500,700,800,np.nan,np.nan,np.nan,np.nan]
    # if the fly's future path intersected only plumes 2,3,4, at those distances) )

    #Using geometric algorithm here: http://geomalgorithms.com/a05-_intersect-1.html
    #referring to diagram under 'Non-Parallel Lines'

    #u - vector from source (P_0) downwind
    #v - heading vector originating from release location (Q_0)
    #w - vector from release location (Q_0) to source (P_0)
    #s_1 - distance along u from source (P_0) to intersection point
    #t_1 - distance along v from release_location (Q_0) to intersection point

    #Turn the track heading angles into unit vectors (v)
    fly_unit_vectors = np.vstack((np.cos(track_heading_angles),np.sin(track_heading_angles)))

    #shape is (2 x flies)
    #Turn (wind_angle-cone_angle,wind_angle+cone_angle) into a plume unit vector (u)
    plume_unit_vector = np.array(
        [[np.cos(wind_angle-cone_angle),np.sin(wind_angle-cone_angle)],
        [np.cos(wind_angle+cone_angle),np.sin(wind_angle+cone_angle)]])

    #shape is (cone sides (2) x 2)

    #Compute vector from release location to each source
    w = source_locations - release_location
    #shape is (traps x 2)

    #Compute s_1 & t_1

    #In the none-wedge version, the below have shape (traps x 2 x flies), and
    #then collapse to (traps x flies) after the computation.

    #For wedges, add an extra dimension to the front
    #for 'which cone side' (-cone_angle,cone_angle)

    #new shape is (2 x traps x 2 x flies)--> collapse to (2 x traps x flies)

    #s_1 = (v2*w1-v1*w2)/(v1*u2-v2*u1)
    s_1 = (fly_unit_vectors[None,None,1,:]*w[None,:,0,None]
        -fly_unit_vectors[None,None,0,:]*w[None,:,1,None])/(
            fly_unit_vectors[None,None,0,:]*plume_unit_vector[:,None,1,None]-
                fly_unit_vectors[None,None,1,:]*plume_unit_vector[:,None,0,None])

    #t_1 = (u1*w2-u2*w1)/(u1*v2-u2*v1)
    t_1 = (plume_unit_vector[:,None,0,None]*w[None,:,1,None]
        -plume_unit_vector[:,None,1,None]*w[None,:,0,None])/(
            fly_unit_vectors[None,None,1,:]*plume_unit_vector[:,None,0,None]-
                fly_unit_vectors[None,None,0,:]*plume_unit_vector[:,None,1,None])

    #collapse extra axis
    s_1 = np.squeeze(s_1) #now shape is (2 x traps x flies)

    #set all intersection values where s_1 or t_1 is negative to np.inf

    inds = (s_1>0.)&(t_1>0.)
    s_1[~inds] = np.inf
    t_1[~inds] = np.inf

  
    #collapse s_1 and t_1 along the 'which cone side' dimension
    #by selecting the cone side with the shorter (finite, positive)
    #t_1 (distance from release_location to intersection point)

    #first collapse t_1
    collapsed_t_1 = np.min(t_1,axis=0)

    where_no_intersections = np.isinf(collapsed_t_1) #value true for fly-trap pairs with no intersection

    #make cone_side_mask, shape (traps x flies) which is a mask (entries (0,1))
    #of which cone sides were intersected (nan if neither)
    cone_side_mask_rough = (t_1==np.min(t_1,axis=0))
    cone_side_mask = np.full(np.shape(collapsed_t_1),np.nan)
    cone_side_mask[cone_side_mask_rough[0]] = 0.
    cone_side_mask[cone_side_mask_rough[1]] = 1.
    cone_side_mask[where_no_intersections] = np.nan

    #now collapse s_1, the distance from source (P_0) to intersection point,
    #by selecting the assigned side (or none of them)

    collapsed_s_1 = np.full(np.shape(collapsed_t_1),np.nan)

    collapsed_s_1[(cone_side_mask==0.)] = s_1[
        0,np.where(cone_side_mask==0)[0],np.where(cone_side_mask==0)[1]]
    collapsed_s_1[(cone_side_mask==1.)] = s_1[
        1,np.where(cone_side_mask==1.)[0],np.where(cone_side_mask==1.)[1]]

    #Check that the shape of s_1 and t_1 are as expected

    return collapsed_s_1.T,collapsed_t_1.T
    #s_1.T - shape is (n flies x n plumes), entry is the distance downwind of that plume
    #t_1.T - shape is (n flies x n plumes),
        #entry is the distance traveled from the release point to the intersection point


def logistic_1d(x,K,x_0,wind_angle):
        Q = 1.
        B = 0.05
        output = np.zeros_like(x)
        output[x>0.]=-K+K/((1.+Q*np.exp(-B*(x[x>0.]-x_0))))
        return output

def f2(intersection_distances,K,x_0,source_locations,wind_angle):
    #Convert intersection_distances to probabilities
    #using a specified distance-probability function

    success_probabilities = logistic_1d(intersection_distances,K,
                                            x_0,wind_angle)

    return success_probabilities

def f3(success_probabilities,dispersal_distances):

    #Use the success_probabilities to determine (with a random draw)
    #which plume each fly will chase, if any


    #the input is shape (n flies x n plumes)

    #First, do a random draw for each plume it will intersect

    #Choose among the positive results of the draw (among possible plumes)
    #by **choosing the plume that drew a 1 which has the shortest dispersal
    #distance for the fly**

    draws = np.random.binomial(1,success_probabilities,
        size=np.shape(success_probabilities)).astype(bool)



    distances_with_positive_draws = np.full_like(success_probabilities,np.nan)

    distances_with_positive_draws[draws] = dispersal_distances[draws]

    #Check that the above value either has positive floats or nan's
    assert(np.sum(distances_with_positive_draws[~np.isnan(
        distances_with_positive_draws)]<0.)<1)

    #Take the inverse of each element to reverse the order and avoid 0 being the min
    distances_with_positive_draws[~np.isnan(
        distances_with_positive_draws)] = 1./(
            distances_with_positive_draws[~np.isnan(distances_with_positive_draws)])

    #Set the nan values to 0.
    distances_with_positive_draws[np.isnan(distances_with_positive_draws)] = 0.


    plume_assignments = distances_with_positive_draws.argmax(axis=1).astype(float)

    plume_assignments[np.sum(draws,axis=1)==0] = np.nan


    #output is of shape (flies) which has for each fly number 0-7 of the traps
    #it goes to, or np.nan if it does not go to a trap
 
    return plume_assignments

def f4(plume_assignments,dispersal_distances,dispersing_speeds):
    #Now that we know which plume each fly is chasing,
    #use the intersection distances and plume ids to compute the time
    #it took for each fly to travel from the release site to
    #the plume it ended up detecting

    release_to_chosen_plume_distances = np.full(len(plume_assignments),np.nan)

    mask = ~np.isnan(plume_assignments)
    cols = plume_assignments[mask].astype(int)
    rows = np.where(mask)

    release_to_chosen_plume_distances[mask] = dispersal_distances[rows,cols].flatten()

    dispersal_travel_times = (release_to_chosen_plume_distances/dispersing_speeds)

    return dispersal_travel_times,release_to_chosen_plume_distances

def f5(plume_assignments,dispersal_travel_times,\
    intersection_distances,fly_speed,release_times):
    #Use the intersection distances and plume ids to compute the time
    #that each fly arrived at the source of the plume it successfully chases
    intersection_distances_chosen_plume = np.full(len(plume_assignments),np.nan)

    mask = ~np.isnan(plume_assignments)
    cols = plume_assignments[mask].astype(int)
    rows = np.where(mask)[0]

    intersection_distances_chosen_plume[mask] = intersection_distances[rows,cols].flatten()

    chasing_times = (intersection_distances_chosen_plume/fly_speed)

    arrival_times = release_times+dispersal_travel_times+chasing_times

    return arrival_times[~np.isnan(arrival_times)],chasing_times,rows,cols


class UpdatingVPatch(object):
    def __init__(self,x_0,width):
        self.rectangle = plt.Rectangle((x_0,0),width,1.,alpha=0.5,color='orange')
    def update(self,new_x_0,new_width):
        self.rectangle.set_x(new_x_0)
        self.rectangle.set_width(new_width)
        # return self.rectangle

def plot_wedges(source_pos,wind_angle,cone_angle):
    length = 2000
    first_arms = np.array(
        (source_pos[:,0]+length*np.cos(wind_angle+cone_angle),
        source_pos[:,1]+length*np.sin(wind_angle+cone_angle))).T
    second_arms = np.array(
        (source_pos[:,0]+length*np.cos(wind_angle-cone_angle),
        source_pos[:,1]+length*np.sin(wind_angle-cone_angle))).T
    #shape traps x 2
    merged = np.stack((first_arms,source_pos,second_arms))
    return merged

In [3]:
#Display setup
num_flies = 2000
fly_speed = 1.6

number_sources = 8
radius_sources = 1000.0
source_locations, _ = create_circle_of_sources(number_sources,
                radius_sources,None)
source_pos = scipy.array([scipy.array(tup) for tup in source_locations])
release_location = np.zeros(2)

intended_heading_angles = np.random.uniform(0,2*np.pi,num_flies)
intended_heading_angles = np.linspace(0,2*np.pi,num_flies)

initial_cone_angle = np.radians(10.)


style = {'description_width': 'initial'}



windmag_slider = widgets.FloatSlider(
    value=1.5,
    min=0,
    max=2.0,
    step=0.1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    description='wind_mag',
    readout=True,
    readout_format='.1f',
    style=style
)

cone_angle_slider = widgets.FloatSlider(
    value=np.degrees(initial_cone_angle),
    min=0.,
    max=25.0,
    step=1.,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    description='cone_angle',
    readout=True,
    readout_format='.1f',
    style=style
)

x_0_slider = widgets.FloatSlider(
    value=300.,
    min=150.,
    max=1000.0,
    step=10.,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    description='x_0',
    readout=True,
    readout_format='.1f',
    style=style
)

radio = widgets.RadioButtons(options=[('Headings',0), ('Intersections',1)],disabled=False,description='toggle')                                 



def slider(start,stop,step,init,description):
    return widgets.FloatSlider(
    value=init,
    min=start,
    max=stop,
    step=step,
    disabled=False,
    continuous_update=False,
    description=description,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style=style
)

# def f(wind_mag,toggle,cone_angle,K,x_0,x_0a,x_0b,L,y_0,t)
def f(wind_mag,toggle,cone_angle,x_0,x_0a,x_0b,t):

    c1=0
    c2=1
    beta=1
    
    #plot scaffolding
    global fig,gs
    gs = GridSpec(9, 16)
    fig = plt.figure(1,figsize=(17,10))
    
    
    #----OBTAIN NEW VALUES--------
    wind_angle = 7*scipy.pi/8.
    cone_angle = np.radians(cone_angle)

    release_times=np.random.exponential(60*beta,num_flies)
    
    plt.figure(1)
               

    K = -1*0.4
    y_0 = 1.6
    L = 0.1
    
    
    #------RECOMPUTE OUTPUTS------
    track_heading_angles,dispersing_speeds = f0(intended_heading_angles,wind_mag,
        wind_angle,x_0a,x_0b,L,y_0,plot=False,c1=c1,c2=c2)
    intersection_distances,dispersal_distances = f1_wedge(
        track_heading_angles,source_pos,wind_angle,cone_angle)
    success_probabilities = f2(intersection_distances,K,x_0,source_pos,wind_angle)
    plume_assignments = f3(success_probabilities,dispersal_distances)
    dispersal_travel_times,release_to_chosen_plume_distances = f4(
            plume_assignments,dispersal_distances,dispersing_speeds)
    arrival_times,chasing_times,\
    which_flies,which_traps = f5(plume_assignments,dispersal_travel_times,
        intersection_distances,fly_speed,release_times)

  
    #------"FIGURE" 0: groundspeed curve
    ax =  fig.add_subplot(gs[2:7,10:])   
    ax.set_aspect('equal')
    ax.spines['left'].set_position('center')
    ax.spines['bottom'].set_position('center')

    # Eliminate upper and right axes
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')

    # Show ticks in the left and lower axes only
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')

    ax.spines['bottom'].set_position('zero')
    ax.spines['left'].set_position('zero')

    ax.set_xlim([-1,7])
    ax.set_ylim([-1,6])
    ax.text(0.7,-0.05,'(Parallel) Windspeed',transform=ax.transAxes)
    ax.text(-0.05,0.7,'Groundspeed',transform=ax.transAxes,rotation=90)

    inputs = np.linspace(-4,7,100)
    plt.plot(inputs,speed_sigmoid_func_kwarg(inputs,x_0a,x_0b,L,4.,y_0),color='orange')
    plt.plot(x_0a*np.ones(100),np.linspace(-1,6,100),color='navy')
    ax.text(x_0a-0.5,3.,'x_0a',horizontalalignment='center',transform=ax.transData,color='navy')

    plt.plot(x_0b*np.ones(100),np.linspace(-1,6,100),color='navy')
    ax.text(x_0b+0.5,3.,'x_0b',horizontalalignment='center',transform=ax.transData,color='navy')

    
    #------"FIGURE" 1 
    ax =  fig.add_subplot(gs[0:4,0:4])   
    xlim = (-1500., 1500.)
    ylim = (-1500., 1500.)
    im_extents = xlim[0], xlim[1], ylim[0], ylim[1]

    ax.set_ylim(list(ylim))
    ax.set_xlim(list(xlim))
    
        
    wedge_points = plot_wedges(source_pos,wind_angle,cone_angle)

    plume_wedges = [matplotlib.patches.Polygon(
        wedge_points[:,i,:],color='black',alpha=0.2) for i in range(number_sources)]

    for plume_wedge in plume_wedges:
        ax.add_patch(plume_wedge)

    for x,y in source_locations:
        ax.scatter(x,y,marker='x',s=50,c='k')
    
    ax.set_aspect('equal')
          
#     ax.set_xticks([])
#     ax.set_yticks([])
    

    
    if toggle==1:
        x,y = release_to_chosen_plume_distances*np.cos(track_heading_angles), \
            release_to_chosen_plume_distances*np.sin(track_heading_angles)
        
    if toggle==0: 
        time = t*60.
        mag = time*dispersing_speeds 
        x,y = mag*np.cos(track_heading_angles), \
            mag*np.sin(track_heading_angles)
     
    if toggle<2:
        fly_dots = plt.scatter(x,y,alpha=0.5,c=dispersing_speeds,vmin=0.,vmax=4.,cmap='cool')
        n = matplotlib.colors.Normalize(vmin = 0., vmax = 4.)
        m = matplotlib.cm.ScalarMappable(norm=n, cmap='cool')
        fly_dots.set_facecolor(m.to_rgba(dispersing_speeds))
        fly_dots.set_clim(vmin=0., vmax=4.)
                
        axins = inset_axes(ax,
            width="50%",  # width = 50% of parent_bbox width
            height="5%",  # height : 5%
            bbox_to_anchor=(-0.25,-0.55, 1, 0.5),
            bbox_transform=ax.transAxes,
            borderpad=0)
        
        fig.colorbar(fly_dots,cax=axins,pad=0.2,orientation='horizontal')         
        
        ax.text(0.5,-0.2,'Dispersal Speed (m/s)',horizontalalignment='center',
            verticalalignment='center',fontsize=12,transform=ax.transAxes)

    

    elif toggle==2:
#         sign = np.sign(dispersing_speeds)
#         time = sign*t*60.
        rad = 500.
        n_bins = 2**6
        bins = np.linspace(0,2*np.pi,n_bins)
        
        
        n,bins = np.histogram(track_heading_angles%(2*np.pi),bins=bins)
        
        center_rs = 0*rad*np.ones_like(bins[:-1])
        n_scale = rad/np.max(n)
        outside_rs = center_rs + n_scale*n
        inside_rs = center_rs - n_scale*n
        
#         plt.plot(center_rs*np.cos(bins[:-1]),center_rs*np.sin(bins[:-1]),color='orange')
        plt.plot(outside_rs*np.cos(bins[:-1]),outside_rs*np.sin(bins[:-1]),color='orange')
#         plt.plot(inside_rs*np.cos(bins[:-1]),inside_rs*np.sin(bins[:-1]),color='orange')
        
    
        rad2 = 700
        density_dots = plt.scatter(rad2*np.cos(bins[:-1]),rad2*np.sin(bins[:-1]),
                                   c=n/(num_flies/n_bins),vmin=0,vmax=(np.max(n))/(num_flies/n_bins))

        axins = inset_axes(ax,
            width="50%",  # width = 50% of parent_bbox width
            height="5%",  # height : 5%
            loc='lower center',
            bbox_to_anchor=(-0.25, -0.55, 1, .5),
            bbox_transform=ax.transAxes,
            borderpad=0)

        
        fig.colorbar(density_dots,cax=axins,pad=0.4,orientation='horizontal')  
        
        ax.text(0.5,-0.3,'Directional Density',horizontalalignment='center',
         verticalalignment='center',fontsize=12,transform=ax.transAxes)


        
    ax.set_xticks([])
    ax.set_yticks([])
    

    #------------"FIGURE" 2 : ARRIVAL CDFs---------------# 
    num_bins = 50

    trap_counts = scipy.zeros(8)
    rasters = []
    labels = ['N','NE','E','SE','S','SW','W','NW']
    sim_reorder = scipy.array([3,2,1,8,7,6,5,4])
    axes = []
    lines = []
    cdf_patches = []
    cdf_steepnesses = np.zeros(8)
    first_hit_times = np.full(8,np.nan)
    new_maxes = num_flies/50.*np.ones(8)
    for i in range(8):

        row = sim_reorder[i]-1
        ax =  fig.add_subplot(gs[row,5:9])    
        t_sim = arrival_times[which_traps==i]

        if len(t_sim)==0:
            ax.set_xticks([0,10,20,30,40,50])
            trap_total = 0
            patch_object = UpdatingVPatch(50,50)
            ax.add_patch(patch_object.rectangle)
            cdf_patches.append(patch_object)
            pass
        else:
            t_sim = t_sim/60.
            (n, bins) = np.histogram(t_sim,bins=num_bins,
                range=(0,max(t_sim)))
            cum_n = np.cumsum(n)
            line, = plt.step(bins,np.hstack((np.array([0,]),cum_n)))
            lines.append(line)
 
            patch_object = UpdatingVPatch(min(t_sim),max(t_sim)-min(t_sim))
            ax.add_patch(patch_object.rectangle)
            cdf_patches.append(patch_object)
            try:
                trap_counts[i]=max(cum_n)
            except(IndexError):
                trap_counts[i]=0

            cdf_steepnesses[i] = trap_counts[i]/(max(t_sim)-min(t_sim))
            first_hit_times[i] = min(t_sim)
            new_maxes[i] = max(num_flies/50.,50*np.ceil(max(cum_n)/50.))
    


        if sim_reorder[i]-1==0:
             ax.set_title('Cumulative Trap Arrivals')

        ax.set_xlim([0,50])
        plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=True,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=True)
        ax.text(1.1,0.5,str(labels[sim_reorder[i]-1]),transform=ax.transAxes,fontsize=15,
            horizontalalignment='center',verticalalignment='center')
        if sim_reorder[i]-1==7:
            ax.set_ylabel('Flies Arrived',x=0.5,horizontalalignment='center',fontsize=12)
            plt.tick_params(axis='y', which='major', labelsize=15)
            ax.set_xlabel('Time (min)',x=0.5,horizontalalignment='center',fontsize=12)
            plt.tick_params(axis='x', which='major', labelsize=15)

        else:
            ax.set_xticklabels('')
        axes.append(ax)
   
    for i,ax in enumerate(axes):
        ax.set_yticks((num_flies/100)*np.array([0.,1.,2.,3.,4.]))
        ax.set_ylim([0,np.max(new_maxes)])
        patch_object = cdf_patches[i]
        try:
            patch_object.rectangle.set_height(ax.get_ylim()[1])
    
        except(ValueError):
            pass
    
    
  
    #------------"FIGURE" 3 : Trap Histograms---------------# 
    
    ax =  fig.add_subplot(gs[4:8,0:4])   
    ax.set_aspect('equal')
    steepness_max = 1.5/100.*num_flies

    num_traps = np.shape(source_pos)[0]
    trap_locs = (2*np.pi/num_traps)*np.array(range(num_traps))
    #Set 0s to 1 for plotting purposes
    trap_counts[trap_counts==0] = .5
    radius_scale = 0.3
    plot_size = 1.5
    trap_locs_2d = [(scipy.cos(trap_loc),scipy.sin(trap_loc)) for trap_loc in trap_locs]
#     trap_patches = [plt.Circle(center, size,
#         alpha=min(cdf_steepnesses[i]/steepness_max,1.)) for center, size, i in zip(
#             trap_locs_2d, radius_scale*trap_counts/np.max(new_maxes),range(8))]
    trap_patches = [plt.Circle(center, size) for center, size, i in zip(
        trap_locs_2d, radius_scale*trap_counts/np.max(new_maxes),range(8))]

    for trap_patch in trap_patches:
        ax.add_patch(trap_patch)

    vmin = 5.;vmax = 20.
    trap_cmap_vals = (first_hit_times-vmin)/vmax
    trap_cmap  = matplotlib.cm.get_cmap('winter')
  
    for trap_cmap_val,trap_patch in zip(trap_cmap_vals,trap_patches):
        # trap_patch.set_color(trap_cmap(trap_cmap_val)[:-1])
        color = tuple(np.array((trap_cmap(trap_cmap_val)[:-1])).astype(float).tolist())
        trap_patch.set_color(color)

    ax.set_ylim([-plot_size,plot_size]);ax.set_xlim([-plot_size,plot_size])
    ax.set_xticks([])
    ax.set_xticklabels('')
    ax.set_yticks([])
    ax.set_yticklabels('')

    coll = matplotlib.collections.PatchCollection(trap_patches)#, facecolors=colors,edgecolors=colors)
    coll.set(cmap=trap_cmap,array=[])
    coll.set_clim(vmin=vmin,vmax=vmax)
        
    axins = inset_axes(ax,
                    width="50%",  # width = 50% of parent_bbox width
                    height="5%",  # height : 5%
                    loc='lower center',
                    bbox_to_anchor=(0.0,-0.2, 1, 0.5),
                    bbox_transform=ax.transAxes,
  )

    fig.colorbar(coll, cax=axins,pad=0.2,orientation='horizontal')

    
    ax.text(0.5,-0.3,'First Arrival Time (min)',horizontalalignment='center',
        verticalalignment='center',fontsize=12,transform=ax.transAxes)
    
    
    #Wind arrow
    plt.arrow(0.5, 0.5, 0.1*scipy.cos(wind_angle), 0.1*scipy.sin(wind_angle),transform=ax.transAxes,color='b',
        width=0.01,head_width=0.05)
    fontsize=15
    ax.text(0,1.5,'N',horizontalalignment='center',verticalalignment='center',fontsize=fontsize)
    ax.text(0,-1.5,'S',horizontalalignment='center',verticalalignment='center',fontsize=fontsize)
    ax.text(1.5,0,'E',horizontalalignment='center',verticalalignment='center',fontsize=fontsize)
    ax.text(-1.5,0,'W',horizontalalignment='center',verticalalignment='center',fontsize=fontsize)
    ax.axis('off')

    #Tidy up plot
    plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9,wspace=0.5,hspace=0.5)

x_0a =slider(-2,6,0.1,-0.4,'x_0a')
x_0b =slider(-2,6,0.1,1.45,'x_0b')
t=slider(0,30,1,5,'t')
#radio = slider(0,30,1,5,'toggle')
    
sliders = windmag_slider, cone_angle_slider, x_0_slider,radio,x_0a,x_0b,t
slider_names = [slider.description for slider in sliders]
param_dict =  dict(zip(slider_names,sliders))
out_box = Box([widgets.interactive_output(f, param_dict)],  layout=Layout(
    display='flex', flex_flow='column',
    border='solid 2px', align_items='stretch',
    width='100%'))

#Rename the slider.description for visual clarity
slider_descriptions = ['Wind Speed','Plume Thickness','Plume Detection Range','x_0a','x_0b','Time']

for slider,slider_description in zip([windmag_slider, cone_angle_slider, x_0_slider,x_0a,x_0b,t],slider_descriptions):
    slider.description = slider_description
    
radio.description = ''
    
items = [Box([slider]) for slider in sliders]
ui_box = Box(items, layout=Layout(
    display='flex', flex_flow='column',
    align_items='stretch', #border='solid 2px',
    width='40%'))



display(VBox([out_box,ui_box]))
#display(GridBox([out_box,ui_box],layout=widgets.Layout(grid_template_columns="repeat(2, 500px)")))


Toggle "Headings/Intersections" to switch between seeing how the flies depart from the release point, to seeing where they intersect the plumes.

Drag "Time" to see how the front of flies expands.

x_0a and x_0b describe the lowest and highest windspeed at which the flies maintain perfect groundspeed control.